In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from models import FineTuneBaseModel
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 6, 2, 17, 16, 14),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [13]:
print(db.query("SELECT POST_ID FROM POSTS WHERE TARGET IN (0, 1)"))

[('nnpn9z',), ('nns6mk',), ('nn4aej',), ('nmpk6k',), ('nmjo2l',), ('nmabst',), ('nm53o9',), ('nmthag',), ('nmpq6d',)]


In [13]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 31, 18, 8, 14))]


In [8]:
db.query('''SELECT TITLE, TARGET FROM POSTS;''')

[('AMC & GME Price Action Comparison before their squeeze', -1),
 ('7k Loss on GME 0DTE', -1),
 ('GME & AMC', -1),
 ('AMC GME Meme 🚀🌕', -1),
 ('Proper 401k weighting for a 26 year old? 99.8% $GME, 0.2% $AMC', -1),
 ('Happy to see everyone make it all back on GME and AMC... Anyone still holding BB like a retard? Make',
  -1),
 ('For all of you retarded apes talking about the $GME $AMC short squeezing to the millions',
  -1),
 ('Welp. I’ve been selling CC’s on my GME to bring my cost basis down since January. I knew this day wo',
  -1),
 ('What if GME does a OSTK.', -1),
 ("I am an idiot. Wife doesn't know yet. Yolo'd into GME at the peak hoping to correct this loss.",
  -1)]

In [27]:
db.query('''SELECT COMMENT, TARGET FROM COMMENTS;''')

[('hell yeah! now give me some coke.', 1),
 ("Look, I don't even have a position in AMC anymore, but it's pretty clear this shit isn't over. Next week is gonna get crazy.",
  1),
 ('That was aggressive', -1),
 ("Boys got a touch of the 'tardation", -1),
 ('I’m not Fucking selling', -1),
 ("With my crumb of the crop $525 buying in at $12.04....I'm holding till I can give my doggo the acreage he deserves yooo!!! Sploooge",
  -1),
 ('This is my hat now, totally my hat!!', -1),
 ('Hilarious movie btw', -1),
 ("Haha, I'm always impressed with 20s something with this much cash. I'm 44 and had nothing like savings until mid 30s. Bravo",
  -1),
 ('Looks like you have unallocated cash?', -1),
 ('I mean, obviously this was getting removed\n\nEdit: changed our minds', -1),
 ('GME is NYSE....', -1),
 ('I am still holding BB w $20 average. Help me', -1),
 ('BB HOLDING STRONG!', -1),
 ("But they are covered calls right?  You aren't down.  You are up.", -1),
 ("I rolled up and out.  I wrote 5/28 $200

In [39]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}' AND STOCK_ID='{}'".format(indexes[0][0], 'GME'))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [62]:
db.query("SELECT COUNT(*) FROM POSTS WHERE StOCK_ID='{}';".format("GME"))[0][0]

10

In [17]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 10, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 5, 31, 18, 50, 59),)]
connection established
[(datetime.datetime(2021, 5, 31, 18, 50, 59),)]


  0%|          | 0/10 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 5, 31, 18, 51, 1),)]


100%|██████████| 10/10 [01:54<00:00, 11.49s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [8]:
print(db.query('show tables;'))

[('POSTS',)]


In [5]:
print(db.query('''SELECT POST_ID FROM POSTS WHERE TARGET=-1 ;'''))

[('nnpn9z',), ('nn4aej',), ('nns6mk',), ('nmpk6k',), ('nmjo2l',), ('nm53o9',), ('nmabst',), ('nmthag',), ('nmmu24',), ('nmpq6d',)]


In [4]:
print(db.query('''SHOW COLUMNS FROM LABELED_POSTS;'''))

[('POST_ID', b'char(20)', 'NO', '', None, ''), ('TARGET', b'int', 'YES', '', None, ''), ('TITLE', b'char(100)', 'YES', '', None, '')]


In [4]:
db.drop_table("LABELED_POSTS")
db.drop_table("LABELED_COMMENTS")

In [3]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [2]:
indices = get_indices("no_stock_id")

connection established
[(datetime.datetime(2021, 6, 2, 17, 22, 23),)]


In [3]:
post_train = PostDataset(512, indices['post_train'])



connection established
[(datetime.datetime(2021, 6, 2, 17, 22, 24),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
sample


{'post': ['GME & AMC',
  'What if GME does a OSTK.',
  'Proper 401k weighting for a 26 year old? 99.8% $GME, 0.2% $AMC'],
 'post_input_ids': tensor([[  101, 14748,  2036,  ...,     0,     0,     0],
         [  101,  1327,  1191,  ...,     0,     0,     0],
         [  101,  5096,  3365,  ...,     0,     0,     0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([[[0., 1.]],
 
         [[1., 0.]],
 
         [[0., 1.]]])}

In [9]:
sample['target'].reshape(-1,2)

tensor([[0., 1.],
        [1., 0.],
        [0., 1.]])

In [14]:
import torch 
_, ground_truths = torch.max(sample['target'].reshape(-1,2), dim=1)

In [15]:
ground_truths

tensor([1, 0, 1])

In [16]:
model = FineTuneBaseModel()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'torch.Tensor'>
tensor([[-0.6015,  0.3814,  0.9997,  ...,  0.9999, -0.6898,  0.9579],
        [-0.6080,  0.4887,  0.9999,  ...,  1.0000, -0.9328,  0.9946],
        [-0.5415,  0.3671,  0.9996,  ...,  0.9998, -0.7365,  0.9712]],
       grad_fn=<TanhBackward>)


In [17]:
out

tensor([[0.7030, 0.2970],
        [0.7028, 0.2972],
        [0.7087, 0.2913]], grad_fn=<SoftmaxBackward>)

In [20]:
_, preds = torch.max(out, dim=1)
preds

tensor([0, 0, 0])

In [23]:
torch.sum(preds==ground_truths)/len(preds)

tensor(0.3333)

In [10]:
out.size()

torch.Size([3, 2])

In [9]:
import torch
torch.Tensor([1, 0]).reshape(1,2).size()

torch.Size([1, 2])

In [8]:
label = torch.Tensor([torch.Tensor([0, 1]).reshape(1,2)]

ValueError: only one element tensors can be converted to Python scalars